# Using Curves with an Index and Inflation Instruments

This page will exemplify the ways of constructing *Curves* dealing with inflation and products that need those curves.
E.g. IndexFixedRateBonds and ZCIS.

### Begin with a simple case without an Index Curve

This case uses an `IndexFixedRateBond` which has **two** coupon periods. The first coupon period was paid in the past so it is no longer relevant, and the second coupon period is paid in the future but the inflation numbers have already been published so its final cashflow.

The bond that is created below is fictional. It has the normal 3 month index lag, *'daily'* index interpolation and the base index for the *Instrument* is set to 101.0

In [13]:
from rateslib import *

today = dt(2025, 5, 12)
ukti = IndexFixedRateBond(
    effective=dt(2024, 5, 27),
    termination=dt(2025, 5, 27),
    fixed_rate=2.0,
    notional=-10e6,
    index_base=101.00,
    index_method="daily",
    index_lag=3,
    spec="uk_gb"
)

In [14]:
ukti.cashflows()

,Type,Period,Ccy,Acc Start,Acc End,Payment,Convention,DCF,Notional,DF,...,Rate,Spread,Real Cashflow,Index Base,Index Val,Index Ratio,Cashflow,NPV,FX Rate,NPV Ccy
0,IndexFixedPeriod,Regular,GBP,2024-05-27,2024-11-27,2024-11-27,actacticma,0.5,-10000000.0,None,...,2.0,NaN,100000.0,101.0,None,None,None,None,1.0,None
1,IndexFixedPeriod,Regular,GBP,2024-11-27,2025-05-27,2025-05-27,actacticma,0.5,-10000000.0,None,...,2.0,NaN,100000.0,101.0,None,None,None,None,1.0,None
2,IndexCashflow,Exchange,GBP,NaT,2025-05-27,2025-05-27,NaN,NaN,-10000000.0,None,...,NaN,NaN,10000000.0,101.0,None,None,None,None,1.0,None


The **cashflows** above are not fully formed. The actual, indexed, cashflows are not known because the relevant index values have not been supplied and there is no *Curve* from which to forecast them.

In this case this bond is linked to UK-RPI. Released values for a month are attributed to the **start** of that month and therefore, as of today's date we are aware of the following, relevant, inflation publications.

In [23]:
from pandas import DataFrame
RPI = DataFrame([
    [dt(2024, 2, 1), 381.0],
    [dt(2024, 3, 1), 383.0],
    [dt(2024, 4, 1), 385.0],
    [dt(2024, 5, 1), 386.4],
    [dt(2024, 6, 1), 387.3],
    [dt(2024, 7, 1), 387.5],
    [dt(2024, 8, 1), 389.9],
    [dt(2024, 9, 1), 388.6],
    [dt(2024, 10, 1), 390.7],
    [dt(2024, 11, 1), 390.9],
    [dt(2024, 12, 1), 392.1],
    [dt(2025, 1, 1), 391.7],
    [dt(2025, 2, 1), 394.0],
    [dt(2025, 3, 1), 395.3]
], columns=["month", "rate"]).set_index("month")["rate"]
RPI

month
2024-02-01    381.0
2024-03-01    383.0
2024-04-01    385.0
2024-05-01    386.4
2024-06-01    387.3
2024-07-01    387.5
2024-08-01    389.9
2024-09-01    388.6
2024-10-01    390.7
2024-11-01    390.9
2024-12-01    392.1
2025-01-01    391.7
2025-02-01    394.0
2025-03-01    395.3
Name: rate, dtype: float64

In [30]:
%timeit RPI.is_monotonic_increasing

18.7 μs ± 511 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [31]:
%timeit RPI.is_unique

20.4 μs ± 1.16 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [32]:
RPI.is_unique

True

In [33]:
RPI.bastard = True

In [34]:
%timeit RPI.bastard

18 ns ± 0.363 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


We can re-create the bond, this time including the ``index_fixings`` and and obtain more structured results.

In [24]:
ukti = IndexFixedRateBond(
    effective=dt(2024, 5, 27),
    termination=dt(2025, 5, 27),
    fixed_rate=2.0,
    notional=-10e6,
    index_base=101.00,
    index_method="daily",
    index_lag=3,
    index_fixings=RPI,
    spec="uk_gb"
)
ukti.cashflows()

,Type,Period,Ccy,Acc Start,Acc End,Payment,Convention,DCF,Notional,DF,...,Rate,Spread,Real Cashflow,Index Base,Index Val,Index Ratio,Cashflow,NPV,FX Rate,NPV Ccy
0,IndexFixedPeriod,Regular,GBP,2024-05-27,2024-11-27,2024-11-27,actacticma,0.5,-10000000.0,None,...,2.0,NaN,100000.0,101.0,391.94,3.880594,388059.405941,None,1.0,None
1,IndexFixedPeriod,Regular,GBP,2024-11-27,2025-05-27,2025-05-27,actacticma,0.5,-10000000.0,None,...,2.0,NaN,100000.0,101.0,NaN,NaN,NaN,None,1.0,None
2,IndexCashflow,Exchange,GBP,NaT,2025-05-27,2025-05-27,NaN,NaN,-10000000.0,None,...,NaN,NaN,10000000.0,101.0,NaN,NaN,NaN,None,1.0,None
